In [4]:
import os
import utils
import numpy as np
import pandas as pd
from Bio import SeqIO
pj = os.path.join

In [2]:
path='/home/ZwZ/database/M'

## read fasta , M1 and M2 sepeartely

In [7]:
M1 = utils.read_fasta(pj(path,'linsi','M1_all_linsi.fasta'),True)
M2 = utils.read_fasta(pj(path,'linsi','M2_all_linsi.fasta'),True)

### the np array of seq

In [9]:
M1_Seq = np.array([SeqRecord.seq for SeqRecord in M1])
M2_Seq = np.array([SeqRecord.seq for SeqRecord in M2])

## slash statistic of M1

In [26]:
slash_posi=np.where(M1_Seq == '-')

In [27]:
count_freq(slash_posi[1])

,item,frequency
0,0,4
1,1,3
2,251,310


## slash statistic of M2

In [30]:
slash_posi=np.where(M2_Seq == '-')

In [31]:
count_freq(slash_posi[1])

,item,frequency
0,0,4
1,1,3
2,91,310
3,92,310
4,93,857
5,94,869
6,95,874
7,96,891


so,if we trim sequence from site '91'

In [32]:
slash_posi=np.where(M2_Seq[:,:91] == '-')

In [33]:
count_freq(slash_posi[1])

,item,frequency
0,0,4
1,1,3


looks pretty good

In [39]:
len(M1[0].seq),len(M1[0].seq[:251])

(252, 251)

In [40]:
len(M2[0].seq),len(M2[0].seq[:91])

(97, 91)

## combine

In [41]:
from Bio import Seq

In [44]:
con = []
for SeqRecord_1 in M1:
    for SeqRecord_2 in M2:
        if SeqRecord_1.id == SeqRecord_2.id:
            SeqRecord_1.seq = Seq.Seq(SeqRecord_1.seq[:251].__str__() + SeqRecord_2.seq[:91].__str__())
            SeqRecord_1.gene = 'M1+M2'
            con.append(SeqRecord_1)

In [94]:
comb_info=pd.DataFrame([['M1',0,250],['M2',251,341]],columns=['gene','start','end'])

In [61]:
SeqIO.write(con,pj(path,'linsi','M1_M2_all_trim.fasta'),'fasta')

2184

In [60]:
comb_info.set_index('gene').to_csv(pj(path,'linsi','M1_M2_all_trim_combination.csv'))

## Retrieve site

In [112]:
import importlib as imp

In [113]:
imp.reload(utils)
from utils import *

In [130]:
fasta = np.array(fasta_to_vec(read_fasta(pj(path,'linsi','M1_M2_filtered_trim.fasta'))))

#      == discard identical sites across all samples == 

Seq = np.array([SeqVector.seq for SeqVector in fasta])

# return a list of stable sites 
size = len(Seq)
same = []
for i in range(Seq.shape[1]):
    if np.sum(Seq[:,i] == Seq[0,i]) == size:
        same.append(i)


# the index of sites to keep
keep2raw = [i for i in range(Seq.shape[1]) if i not in same]

In [131]:
len(fasta)

397

In [70]:
del site

2

In [158]:
imp.reload(Model)
import Model

In [162]:
keep2raw=sites_to_keep(fasta)

In [163]:
SV_ls=joblib.load(pj('example1','the_3.ModelList'))

In [164]:
omega=Model.SVM_omega(SV_ls[6].coef_,keep2raw=keep2raw)

In [165]:
DF=omega.DF

In [166]:
DF.loc[:,'genes']=DF.raw_sites.apply(lambda x: sites2gene(x,comb_info))
DF.loc[:,'genessites']=DF.raw_sites.apply(lambda x: sites2genesite(x,comb_info))

In [155]:
comb_info = pd.read_csv(pj(path,'linsi','M1_M2_all_trim_combination.csv'))